In [8]:
from sed2 import sed_process_registry, ports, annotate, register
from sed2.core import (
    serialize_instance, generate_script, deserialize_instance,
    register_functions, generate_composite_process, get_process_schema, 
)
from bigraph_viz import plot_bigraph, plot_flow, pf
import numpy as np

# add sed processes to global namespace
sed_process_registry.activate_all(globals())
# print(pf(sed_process_registry.registry))

In [2]:
@register(sed_process_registry)
@ports({
    'outputs': {
        'value': 'float'}})
@annotate('sed:loop')
def random_number():
    return random.random()

# add sed processes to global namespace
# print(pf(process_registry.registry))

In [5]:
def initialize_process_from_schema(schema, process_registry):
    schema = deserialize_instance(schema)
    assert len(schema) == 1 # only one top-level key
    process_name = next(iter(schema))
    process_entry = process_registry.access(process_name)
    process = process_entry['address']
    config = schema['config']
    return {process_name: process(config)}


def get_processes_states_from_schema(schema, process_registry):
    all_annotations = process_registry.get_annotations()

    # separate the processes and states
    processes = {}
    states = {}
    for name, value in schema.items():
        if isinstance(value, dict) and value.get('wires'):
            processes[name] = value
        else:
            states[name] = value
    return processes, states


class Process:
    config = {}
    
    def __init__(self, config):
        self.initialize(config)
        
    def initialize(self, config):
        self.config = config
        
    def ports(self):
        return {}
    
    def update(self, state):
        return {}
    
    
    
class Composite(Process):
    config = {
        'schema': None,
        'process_registry': None,
    }
    
    def __init__(self, config, schema, process_registry):
        self.initialize(config, schema, process_registry)
        
    def initialize(self, config, schema, process_registry):
        self.config = config
        self.schema = schema
        self.process_registry = process_registry
        
        processes, states = get_processes_states_from_schema(
            self.schema, self.process_registry)
        
        self.states = states
        self.processes = {}
        for process in processes:
            initialized_process = initialize_process_from_schema(schema, process_registry)
            self.processes.update(initialized_process)
            
    def process_state(self, process_path):
        # TODO -- get the state from the point of view of the process
        return
    
    def to_json(self):
        return serialize_instance(self.schema)
    
    def run(self):
        return {}
    
 

In [9]:
@register(sed_process_registry)
@ports({
    'inputs': {
        'trials': 'int',
    },
    'outputs': {
        'results': 'list'}})
@annotate('sed:range_iterator')
class RangeIterator(Composite):
    def run(self, trials):
        results = []
        for i in range(trials):
            for process in self.processes:
                # TODO -- get the process state
                result = process.update()
            
        return results
    
    
instance2 = {
    'trials': 10,
    'run': False,
    'loop': {
        '_type': 'sed:range_iterator',
        'wires': {
            'trials': 'trials',
        },
        'value': 0,
        'added': 1,
        'add': {
            '_type': 'math:add',
            'wires': {
                'a': 'value',
                'b': 'added',
                'result': 'value',
            },
        }
    },

}

# fill(instance)  # autocomplete
# validate_schema(instance) # TODO -- demonstrate validation

# Serialize to JSON
json_str1 = serialize_instance(instance2)
print(json_str1)

config = {}
sim_experiment = Composite(config=config, schema=instance2, process_registry=sed_process_registry)

sim_experiment.run()

json_str = sim_experiment.to_json()
print(pf(json_str))

{"trials": 10, "run": false, "loop": {"_type": "sed:range_iterator", "wires": {"trials": "trials"}, "value": 0, "added": 1, "add": {"_type": "math:add", "wires": {"a": "value", "b": "added", "result": "value"}}}}


AssertionError: 

In [ ]:
# # Generate the script
# script = generate_script(json_str1, sed_process_registry)
# print(script)

# # execute the script
# exec(script)

In [ ]:
plot_bigraph(instance2)

## Infix

In [ ]:
import numpy as np
import re

def infix_to_postfix(infix_expression):
    precedence = {'+':1, '-':1, '*':2, '/':2, '^':3}
    right_associative = {'^'}
    stack = [] 
    postfix = []
    
    # Tokenizing the infix expression
    tokens = re.findall(r"[\w.]+|[^ \w]", infix_expression)

    for token in tokens:
        if re.match(r"^[\d.]+$", token):  # If the token is a number (integer or float)
            postfix.append(token)
        elif token.startswith('np'):  # If the token is a numpy array
            postfix.append(token)
        elif token == '(':
            stack.append('(')
        elif token == ')':
            while stack and stack[-1] != '(':
                postfix.append(stack.pop())
            if stack:
                stack.pop()
        else:  # If the token is an operator
            while (stack and stack[-1] != '(' and 
                   ((token not in right_associative and precedence.get(token, 0) <= precedence.get(stack[-1], 0)) or
                   (token in right_associative and precedence.get(token, 0) < precedence.get(stack[-1], 0)))):
                postfix.append(stack.pop())
            stack.append(token)

    while stack:
        postfix.append(stack.pop())

    postfix_expression = " ".join(postfix)
    print(f"Postfix Expression: {postfix_expression}")  # Debugging print statement
    return postfix_expression

def evaluate_postfix(postfix_expression):
    stack = []
    tokens = postfix_expression.split()

    for token in tokens:
        if re.match(r"^[\d.]+$", token):  # If the token is a number (integer or float)
            stack.append(float(token))
        elif token.startswith('np'):  # If the token is a numpy array
            stack.append(eval(token))
        else:  # If the token is an operator
            val1 = stack.pop()
            val2 = stack.pop()
            switcher = {
                '+': val2 + val1,
                '-': val2 - val1,
                '*': val2 * val1,
                '/': val2 / val1,
                '^': val2**val1
            }
            stack.append(switcher.get(token))
    
    print(f"Final Stack: {stack}")  # Debugging print statement
    return stack[0]

def evaluate_infix(infix_expression):
    postfix_expression = infix_to_postfix(infix_expression)
    result = evaluate_postfix(postfix_expression)
    return result


In [ ]:
# # Test the function
# arr1 = np.array([1, 2, 3])
# arr2 = np.array([4, 5, 6])
# print(stringify_array(arr1))  # Output: "array0"
# print(stringify_array(arr2))  # Output: "array1"

# print(evaluate_infix("(22.112+3.123)*0.245^3.1"))